# Brasileirão - 2006 a 2025
>Análises estatísticas dos campeonatos brasileiros na era dos pontos corridos, com 20 times e 38 rodadas.

# Dataprep

# Import

In [1]:
import time
import random
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# plot
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import root_mean_squared_error as rmse

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

# Dataprep


## 2006 - 2024

In [2]:
df_2006_2024 = pd.read_csv('../dados/brasileirao_2006_2024.csv')
df_2006_2024

,ano_campeonato,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante
0,2006,1,Atlético-PR,Fluminense,1.0,2.0
1,2006,1,Botafogo,Fortaleza,1.0,0.0
2,2006,1,Goiás,Santos,0.0,0.0
3,2006,1,Grêmio,Corinthians,2.0,0.0
4,2006,1,Juventude,Paraná,1.0,0.0
...,...,...,...,...,...,...
7215,2024,38,Fortaleza,Internacional,3.0,0.0
7216,2024,38,Grêmio,Corinthians,0.0,3.0
7217,2024,38,Juventude,Cruzeiro,0.0,1.0
7218,2024,38,Palmeiras,Fluminense,0.0,1.0


## 2025

In [3]:
# -------------------------
# Config
# -------------------------
URL = "https://ge.globo.com/futebol/brasileirao-serie-a/"
ANO = 2025
PRIMEIRA_RODADA = 1
ULTIMA_RODADA = 38
OUTPUT_CSV = "../dados/brasileirao_2025.csv"

# -------------------------
# Helpers
# -------------------------
def parse_int_safe(s):
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    m = re.search(r'(\d+)', s)
    return int(m.group(1)) if m else None

def human_pause(a=0.6, b=1.4):
    time.sleep(random.uniform(a, b))

# -------------------------
# Cria driver com stealth manual (sem libs extras)
# -------------------------
def criar_driver_stealth():
    options = webdriver.ChromeOptions()

    # user-agent realista
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/129.0.6668.89 Safari/537.36"
    )
    # reduzir sinais de automação
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window()

    # Injetar script para reduzir sinais de webdriver
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            Object.defineProperty(navigator, 'languages', { get: () => ['pt-BR','pt','en-US'] });
            Object.defineProperty(navigator, 'plugins', { get: () => [1,2,3,4,5] });
            window.chrome = window.chrome || { runtime: {} };
            (function() {
                const getParam = WebGLRenderingContext.prototype.getParameter;
                WebGLRenderingContext.prototype.getParameter = function(param) {
                    if (param === 37445) return 'Intel Inc.';
                    if (param === 37446) return 'Intel Iris OpenGL Engine';
                    return getParam.call(this, param);
                };
            })();
            try {
                Object.defineProperty(window, 'Notification', {
                    get: () => ({ permission: 'granted' })
                });
            } catch(e) {}
        """
    })

    return driver

# -------------------------
# Seletores e operações robustas
# -------------------------
def accept_cookies_if_any(driver, wait):
    selectors = [
        ".cookie-banner-lgpd_accept-button",
        "button.cookie-accept",
        "button[class*='cookie'][class*='accept']",
        "button[data-testid='cookie-accept']",
        "button.cookie-banner__btn"
    ]
    for sel in selectors:
        try:
            btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            human_pause(0.6, 1.2)
            btn.click()
            print("Cookies aceitos pelo seletor:", sel)
            return True
        except Exception:
            continue
    print("Popup de cookies não encontrado ou já aceito.")
    return False

def get_current_round(driver):
    possible = [
        "span.lista-jogos__navegacao--rodada",
        ".lista-jogos__navegacao--rodada",
        "span.navegacao-rodada__numero",
        ".navegacao-rodada__numero"
    ]
    for sel in possible:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            text = el.text.strip()
            m = re.search(r'(\d{1,2})', text)
            if m:
                return int(m.group(1))
        except Exception:
            continue
    # fallback textual
    try:
        body_text = driver.find_element(By.TAG_NAME, "body").text
        m = re.search(r'Rodad[ae]\s*(\d{1,2})', body_text, re.IGNORECASE)
        if m:
            return int(m.group(1))
    except Exception:
        pass
    return None

def click_left(driver, wait):
    selectors = [
        ".lista-jogos__navegacao--seta-esquerda",
        ".lista-jogos__navegacao--setas-ativa.lista-jogos__navegacao--seta-esquerda",
        "button[aria-label*='Anterior']",
        "button[aria-label*='anterior']",
        "button[aria-label*='left']"
    ]
    for sel in selectors:
        try:
            el = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            human_pause(0.4, 1.0)
            el.click()
            return True
        except Exception:
            continue
    return False

def click_right(driver, wait):
    selectors = [
        ".lista-jogos__navegacao--seta-direita",
        ".lista-jogos__navegacao--setas-ativa.lista-jogos__navegacao--seta-direita",
        "button[aria-label*='Próxima']",
        "button[aria-label*='proxima']",
        "button[aria-label*='right']"
    ]
    for sel in selectors:
        try:
            el = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            human_pause(0.4, 1.0)
            el.click()
            return True
        except Exception:
            continue
    return False

# -------------------------
# Função atualizada: extrair jogos usando os seletores fornecidos
# -------------------------
# -------------------------
# Função atualizada com seletores revisados para NOME DO TIME
# -------------------------
# -------------------------
# Função atualizada: extrair jogos usando XPath
# -------------------------
def extrair_jogos_da_rodada(driver, rodada_num):
    """
    Extrai os 10 jogos da rodada usando XPath focado nas classes de time.
    """

    dados = []

    # --- 1) Aguarda o container principal da rodada ---
    try:
        rodada_container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "lista-jogos"))
        )
    except:
        print(f"❌ Rodada {rodada_num}: container '.lista-jogos' não encontrado.")
        return dados

    # --- 2) Cada jogo está dentro de class="lista-jogos__jogo" ---
    jogos = rodada_container.find_elements(By.CLASS_NAME, "lista-jogos__jogo")

    if not jogos:
        print(f"⚠ Rodada {rodada_num}: nenhum elemento '.lista-jogos__jogo' encontrado.")
        return dados

    # --- 3) Extrair dados de cada jogo ---
    for jogo in jogos:

        # ---------- MANDANTE (Nome do time via XPath) ----------
        # TENTATIVA 1: Buscar o nome/sigla dentro do bloco mandante. Priorizamos 'equipes__sigla'.
        xpath_mandante = ".//*[contains(@class, 'placar__equipes--mandante')]/*[contains(@class, 'equipes__sigla') or contains(@class, 'equipes__nome-sigla') or contains(@class, 'equipes__nome')]"
        try:
            mandante = jogo.find_element(
                By.XPATH,
                xpath_mandante
            ).text.strip()
        except Exception:
            mandante = None

        # ---------- VISITANTE (Nome do time via XPath) ----------
        # TENTATIVA 1: Buscar o nome/sigla dentro do bloco visitante. Priorizamos 'equipes__sigla'.
        xpath_visitante = ".//*[contains(@class, 'placar__equipes--visitante')]/*[contains(@class, 'equipes__sigla') or contains(@class, 'equipes__nome-sigla') or contains(@class, 'equipes__nome')]"
        try:
            visitante = jogo.find_element(
                By.XPATH,
                xpath_visitante
            ).text.strip()
        except Exception:
            visitante = None

        # ---------- GOLS MANDANTE (Seletores de placar mantidos por serem mais estáveis) ----------
        try:
            gols_mandante = jogo.find_element(
                By.CSS_SELECTOR,
                ".placar-box__valor.placar-box__valor--mandante"
            ).text.strip()
        except:
            gols_mandante = None

        # ---------- GOLS VISITANTE (Seletores de placar mantidos por serem mais estáveis) ----------
        try:
            gols_visitante = jogo.find_element(
                By.CSS_SELECTOR,
                ".placar-box__valor.placar-box__valor--visitante"
            ).text.strip()
        except:
            gols_visitante = None

        dados.append({
            "ano_campeonato": ANO,
            "rodada": rodada_num,
            "time_mandante": mandante,
            "time_visitante": visitante,
            "gols_mandante": gols_mandante,
            "gols_visitante": gols_visitante
        })

    print(f"✓ Rodada {rodada_num}: {len(dados)} jogos extraídos")
    return dados

# -------------------------
# Fluxo principal (igual ao anterior)
# -------------------------
def main():
    driver = criar_driver_stealth()
    wait = WebDriverWait(driver, 15)
    driver.get(URL)

    # esperar carregar
    time.sleep(2 + random.random())

    # aceitar cookies se houver
    accept_cookies_if_any(driver, wait)

    # detectar rodada inicial (normalmente 35)
    rodada_atual = get_current_round(driver)
    if rodada_atual is None:
        print("Não foi possível detectar rodada inicial; assumindo 35.")
        rodada_atual = 38
    print("Rodada inicial detectada:", rodada_atual)

    # navegar para a rodada 1 clicando à esquerda repetidamente
    attempts = 0
    while rodada_atual is not None and rodada_atual > PRIMEIRA_RODADA and attempts < 60:
        ok = click_left(driver, wait)
        human_pause(0.8, 1.6)
        nova = get_current_round(driver)
        if nova:
            rodada_atual = nova
        attempts += 1
        print(f"Voltando... agora na rodada {rodada_atual} (tent {attempts})")
        if not ok:
            try:
                driver.execute_script("window.scrollBy(0, 200);")
                human_pause(0.4, 0.8)
            except:
                pass

    # garantir estamos na rodada 1 (ou no menor detectado)
    rodada_detalhe = get_current_round(driver) or rodada_atual
    print("Rodada onde iniciaremos extração:", rodada_detalhe)

    all_rows = []

    # Navegar e extrair de 1 até ULTIMA_RODADA
    for target_round in range(PRIMEIRA_RODADA, ULTIMA_RODADA + 1):
        # confirmar rodada exibida e sincronizar (re-try curto)
        shown = get_current_round(driver)
        if shown is None:
            shown = target_round
        print(f"\nExtraindo rodada exibida {shown} (alvo {target_round})")

        # esperar o container da rodada carregar
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "lista-jogos"))
            )
        except:
            print("Atenção: container '.lista-jogos' não carregou rapidamente.")

        human_pause(0.8, 1.4)
        rows = extrair_jogos_da_rodada(driver, shown)
        if not rows:
            print(f"Aviso: nenhum jogo extraído na rodada {shown}.")
        all_rows.extend(rows)

        # se já está na última que queremos, parar
        if shown >= ULTIMA_RODADA:
            break

        # clicar seta direita para avançar
        ok = click_right(driver, wait)
        if not ok:
            print("Falha ao clicar seta direita (tentando scroll e novo clique).")
            try:
                driver.execute_script("window.scrollBy(0, 200);")
                human_pause(0.5, 1.0)
                click_right(driver, wait)
            except:
                print("Não foi possível avançar para próxima rodada por falta de seletor.")
        human_pause(0.8, 1.6)

    # finalizar driver
    try:
        driver.quit()
    except:
        pass

    # montar e salvar dataframe
    df_2025 = pd.DataFrame(all_rows, columns=[
        "ano_campeonato", "rodada", "time_mandante", "time_visitante", "gols_mandante", "gols_visitante"
    ])

    if not df_2025.empty:
        df_2025 = df_2025.sort_values(by=["rodada", "time_mandante"]).reset_index(drop=True)

    df_2025.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\nConcluído. Arquivo salvo: {OUTPUT_CSV}")
    print(df_2025.head())
    return df_2025

if __name__ == "__main__":
    main()


Cookies aceitos pelo seletor: .cookie-banner-lgpd_accept-button
Rodada inicial detectada: 35
Voltando... agora na rodada 34 (tent 1)
Voltando... agora na rodada 34 (tent 2)
Voltando... agora na rodada 34 (tent 3)
Voltando... agora na rodada 34 (tent 4)
Voltando... agora na rodada 34 (tent 5)
Voltando... agora na rodada 33 (tent 6)
Voltando... agora na rodada 32 (tent 7)
Voltando... agora na rodada 31 (tent 8)
Voltando... agora na rodada 30 (tent 9)
Voltando... agora na rodada 29 (tent 10)
Voltando... agora na rodada 28 (tent 11)
Voltando... agora na rodada 27 (tent 12)
Voltando... agora na rodada 26 (tent 13)
Voltando... agora na rodada 25 (tent 14)
Voltando... agora na rodada 24 (tent 15)
Voltando... agora na rodada 23 (tent 16)
Voltando... agora na rodada 22 (tent 17)
Voltando... agora na rodada 21 (tent 18)
Voltando... agora na rodada 20 (tent 19)
Voltando... agora na rodada 19 (tent 20)
Voltando... agora na rodada 18 (tent 21)
Voltando... agora na rodada 17 (tent 22)
Voltando... ag

# Concatenando os df's

In [4]:
df_2025 = pd.read_csv('../dados/brasileirao_2025.csv')

df = pd.concat([df_2006_2024, df_2025])
df.sort_values(['ano_campeonato','rodada','time_mandante'], inplace = True)
df['time_mandante'] = df['time_mandante'].apply(lambda x: x.strip())
df['time_visitante'] = df['time_visitante'].apply(lambda x: x.strip())
print(f'Antes do drop_duplicates: {df.shape}')
df.drop_duplicates(subset = ['ano_campeonato', 'rodada', 'time_mandante', 'time_visitante'], keep = 'last', inplace=True)
print(f'Após o drop_duplicates: {df.shape}')
df

Antes do drop_duplicates: (7600, 6)
Após o drop_duplicates: (7600, 6)


,ano_campeonato,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante
0,2006,1,Atlético-PR,Fluminense,1.0,2.0
1,2006,1,Botafogo,Fortaleza,1.0,0.0
2,2006,1,Goiás,Santos,0.0,0.0
3,2006,1,Grêmio,Corinthians,2.0,0.0
4,2006,1,Juventude,Paraná,1.0,0.0
...,...,...,...,...,...,...
375,2025,38,INT,RBB,NaN,NaN
376,2025,38,MIR,FLA,NaN,NaN
377,2025,38,SAN,CRU,NaN,NaN
378,2025,38,SPT,GRE,NaN,NaN


# Unificando nomes de times similares

In [5]:
dict_times_unificacao = {'Athletico-PR' : 'Atlético-PR',
                         'Goiás EC' : 'Goiás',
                         'Santos FC' : 'Santos',
                         'BAH':'EC Bahia',
                         'CRU':'Cruzeiro',
                         'FLA':'Flamengo',
                         'FOR':'Fortaleza',
                         'GRE':'Grêmio',
                         'JUV':'Juventude',
                         'PAL':'Palmeiras',
                         'RBB':'RB Bragantino',
                         'SAO':'São Paulo',
                         'VAS':'Vasco da Gama',
                         'BOT':'Botafogo',
                         'CAM':'Atlético-MG',
                         'CEA':'Ceará SC',
                         'COR':'Corinthians',
                         'FLU':'Fluminense', 
                         'INT':'Internacional', 
                         'MIR':'Mirassol', 
                         'SAN':'Santos',
                         'SPT':'Sport Recife', 
                         'VIT':'EC Vitória',
                         }

df['time_mandante'].replace(dict_times_unificacao, inplace = True)
df['time_visitante'].replace(dict_times_unificacao, inplace = True)

display(np.sort(df['time_mandante'].unique()))
display(np.sort(df['time_visitante'].unique()))

C:\Users\M4005001\AppData\Local\Temp\ipykernel_14932\1006452405.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['time_mandante'].replace(dict_times_unificacao, inplace = True)
C:\Users\M4005001\AppData\Local\Temp\ipykernel_14932\1006452405.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

array(['América-MG', 'América-RN', 'Atlético-GO', 'Atlético-MG',
       'Atlético-PR', 'Avaí FC', 'Barueri', 'Botafogo', 'CSA', 'Ceará SC',
       'Chapecoense', 'Corinthians', 'Coritiba FC', 'Criciúma EC',
       'Cruzeiro', 'Cuiabá-MT', 'EC Bahia', 'EC Vitória',
       'Figueirense FC', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás',
       'Grêmio', 'Guarani', 'Internacional', 'Ipatinga FC',
       'Joinville-SC', 'Juventude', 'Mirassol', 'Náutico', 'Palmeiras',
       'Paraná', 'Ponte Preta', 'Portuguesa', 'RB Bragantino',
       'Santa Cruz', 'Santo André', 'Santos', 'Sport Recife',
       'São Caetano', 'São Paulo', 'Vasco da Gama'], dtype=object)

array(['América-MG', 'América-RN', 'Atlético-GO', 'Atlético-MG',
       'Atlético-PR', 'Avaí FC', 'Barueri', 'Botafogo', 'CSA', 'Ceará SC',
       'Chapecoense', 'Corinthians', 'Coritiba FC', 'Criciúma EC',
       'Cruzeiro', 'Cuiabá-MT', 'EC Bahia', 'EC Vitória',
       'Figueirense FC', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás',
       'Grêmio', 'Guarani', 'Internacional', 'Ipatinga FC',
       'Joinville-SC', 'Juventude', 'Mirassol', 'Náutico', 'Palmeiras',
       'Paraná', 'Ponte Preta', 'Portuguesa', 'RB Bragantino',
       'Santa Cruz', 'Santo André', 'Santos', 'Sport Recife',
       'São Caetano', 'São Paulo', 'Vasco da Gama'], dtype=object)

# Estatística básica

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ano_campeonato,7600.0,2015.500000,5.766661,2006.0,2010.75,2015.5,2020.25,2025.0
rodada,7600.0,19.500000,10.966578,1.0,10.00,19.5,29.00,38.0
gols_mandante,7567.0,1.485397,1.192991,0.0,1.00,1.0,2.00,8.0
gols_visitante,7567.0,0.998546,1.005338,0.0,0.00,1.0,2.00,7.0


# Testes de consistência dos dados - Parte 1

In [7]:
#rodadas por temporada: esperadas 38 (#ok)
display(df.groupby('ano_campeonato')['rodada'].max())

#jogos por campeonato: esperados 380 (#ok)
display(df.groupby(['ano_campeonato'])['time_mandante'].count())



ano_campeonato
2006    38
2007    38
2008    38
2009    38
2010    38
2011    38
2012    38
2013    38
2014    38
2015    38
2016    38
2017    38
2018    38
2019    38
2020    38
2021    38
2022    38
2023    38
2024    38
2025    38
Name: rodada, dtype: int64

ano_campeonato
2006    380
2007    380
2008    380
2009    380
2010    380
2011    380
2012    380
2013    380
2014    380
2015    380
2016    380
2017    380
2018    380
2019    380
2020    380
2021    380
2022    380
2023    380
2024    380
2025    380
Name: time_mandante, dtype: int64

In [8]:
#jogos como mandante por time por temporada: esperados 19 jogos ao final do campeonato
df_jogos_por_time_mandante = df.groupby(['ano_campeonato','time_mandante']).count()#.reset_index()
display('Jogos por time como mandantes, diferente de 19 jogos:',df_jogos_por_time_mandante.loc[df_jogos_por_time_mandante.rodada!=19])#.sort_values('time_mandante'))

#jogos como visitante por time por temporada: esperados 19 jogos ao final do campeonato
df_jogos_por_time_visitante = df.groupby(['ano_campeonato','time_visitante']).count()#.reset_index()
display('Jogos por time como visitante, diferente de 19 jogos:',df_jogos_por_time_visitante.loc[df_jogos_por_time_visitante.rodada!=19])#.sort_values('time_visitante'))

# Todos os times apresentaram 19 jogos como mandantes e 19 como visitantes por ano, totalizando 38 jogos por campeonato

'Jogos por time como mandantes, diferente de 19 jogos:'

,,rodada,time_visitante,gols_mandante,gols_visitante
ano_campeonato,time_mandante,,,,


'Jogos por time como visitante, diferente de 19 jogos:'

,,rodada,time_mandante,gols_mandante,gols_visitante
ano_campeonato,time_visitante,,,,


# Engenharia de features

In [9]:
df['pontos_mandante'] = df.gols_mandante - df.gols_visitante
df['pontos_mandante'] = df['pontos_mandante'].map(lambda x: 3 if x > 0 else 1 if x == 0 else 0)

df['pontos_visitante'] = df.gols_visitante - df.gols_mandante
df['pontos_visitante'] = df['pontos_visitante'].map(lambda x: 3 if x > 0 else 1 if x == 0 else 0)

df

,ano_campeonato,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante,pontos_mandante,pontos_visitante
0,2006,1,Atlético-PR,Fluminense,1.0,2.0,0,3
1,2006,1,Botafogo,Fortaleza,1.0,0.0,3,0
2,2006,1,Goiás,Santos,0.0,0.0,1,1
3,2006,1,Grêmio,Corinthians,2.0,0.0,3,0
4,2006,1,Juventude,Paraná,1.0,0.0,3,0
...,...,...,...,...,...,...,...,...
375,2025,38,Internacional,RB Bragantino,NaN,NaN,0,0
376,2025,38,Mirassol,Flamengo,NaN,NaN,0,0
377,2025,38,Santos,Cruzeiro,NaN,NaN,0,0
378,2025,38,Sport Recife,Grêmio,NaN,NaN,0,0


In [10]:
#criação de df_mandante e df_visitante para depois concatená-los

#df_mandante
df_mandante = df[['ano_campeonato',	'rodada',	'time_mandante','time_visitante','gols_mandante','gols_visitante']].copy()
df_mandante.rename(columns = {'time_mandante':'time',
                              'time_visitante':'adversário',
                              'gols_mandante':'gols_pro',
                              'gols_visitante': 'gols_contra'}, inplace = True)
df_mandante['pontos'] = df_mandante.gols_pro - df_mandante.gols_contra
df_mandante['pontos'] = df_mandante['pontos'].map(lambda x: 3 if x > 0 else 1 if x == 0 else 0)

#df_visitante
df_visitante = df[['ano_campeonato',	'rodada',	'time_visitante','time_mandante','gols_mandante','gols_visitante']].copy()
df_visitante.rename(columns = {'time_visitante':'time',
                               'time_mandante': 'adversário',
                              'gols_visitante':'gols_pro',
                              'gols_mandante': 'gols_contra'
                              }, inplace = True)

df_visitante['pontos'] = df_visitante.gols_pro - df_visitante.gols_contra
df_visitante['pontos'] = df_visitante['pontos'].map(lambda x: 3 if x > 0 else 1 if x == 0 else 0)

df_concat = pd.concat([df_mandante, df_visitante]).reset_index(drop = True)

#criação das features vitoria, empate, derrota
df_concat['vitoria'] = df_concat['pontos'].map(lambda x: 1 if x == 3 else 0)
df_concat['derrota'] = df_concat['pontos'].map(lambda x: 1 if x == 0 else 0)
df_concat['empate'] = df_concat['pontos'].map(lambda x: 1 if x == 1 else 0)

#anulando dados de rodadas que ainda não aconteceram
df_concat.loc[df_concat.gols_pro != df_concat.gols_pro, ['pontos', 'vitoria', 'derrota', 'empate']] = np.nan

df_concat

,ano_campeonato,rodada,time,adversário,gols_pro,gols_contra,pontos,vitoria,derrota,empate
0,2006,1,Atlético-PR,Fluminense,1.0,2.0,0.0,0.0,1.0,0.0
1,2006,1,Botafogo,Fortaleza,1.0,0.0,3.0,1.0,0.0,0.0
2,2006,1,Goiás,Santos,0.0,0.0,1.0,0.0,0.0,1.0
3,2006,1,Grêmio,Corinthians,2.0,0.0,3.0,1.0,0.0,0.0
4,2006,1,Juventude,Paraná,1.0,0.0,3.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
15195,2025,38,RB Bragantino,Internacional,NaN,NaN,NaN,NaN,NaN,NaN
15196,2025,38,Flamengo,Mirassol,NaN,NaN,NaN,NaN,NaN,NaN
15197,2025,38,Cruzeiro,Santos,NaN,NaN,NaN,NaN,NaN,NaN
15198,2025,38,Grêmio,Sport Recife,NaN,NaN,NaN,NaN,NaN,NaN


# Testes de consistência dos dados - Parte 2

In [11]:
#jogos por temporada: esperados 38 por time (OK)
display(df_concat.groupby(['time','ano_campeonato'])['rodada'].count())
display(df_concat.groupby(['time','ano_campeonato'])['rodada'].count().describe())


#times por temporada: esperados 20
display(df_concat.groupby('ano_campeonato')['time'].nunique())

time            ano_campeonato
América-MG      2011              38
                2016              38
                2018              38
                2021              38
                2022              38
                2023              38
América-RN      2007              38
Atlético-GO     2010              38
                2011              38
                2012              38
                2017              38
                2020              38
                2021              38
                2022              38
                2024              38
Atlético-MG     2007              38
                2008              38
                2009              38
                2010              38
                2011              38
                2012              38
                2013              38
                2014              38
                2015              38
                2016              38
                2017              38
       

count    400.0
mean      38.0
std        0.0
min       38.0
25%       38.0
50%       38.0
75%       38.0
max       38.0
Name: rodada, dtype: float64

ano_campeonato
2006    20
2007    20
2008    20
2009    20
2010    20
2011    20
2012    20
2013    20
2014    20
2015    20
2016    20
2017    20
2018    20
2019    20
2020    20
2021    20
2022    20
2023    20
2024    20
2025    20
Name: time, dtype: int64

# Identificando os campeões de cada ano | 1o turno e final

## Ajuste da base para a rodada#38 de 2016, jogo entre Chapecoense e Atletico MG
* Em função do acidente ocorrido com o avião com o time da Chapecoense em 2016, o jogo da rodada #38 teve WO duplo, ou seja, nenhum dos times compareceram a campo.
* Por tratar-se de uma condição não prevista no regulamento CBF, foi considerada derrota por 3 a 0 para ambos os times.
* Detalhes na matéria: https://www.uol.com.br/esporte/futebol/ultimas-noticias/2016/12/05/presidente-da-chape-diz-que-cbf-ja-cancelou-jogo-contra-o-atletico-mg.htm

* Desta forma a base será ajustada considerando o placar de 0 a 3 para ambos os times.

In [12]:
display('Antes da correção',df_concat.loc[(df_concat.ano_campeonato == 2016) & (df_concat.rodada == 38) & (df_concat.gols_pro.isna())])

df_concat.loc[(df_concat.ano_campeonato == 2016) & (df_concat.rodada == 38) & (df_concat.gols_pro != df_concat.gols_pro), 'gols_contra'] = 3
df_concat.loc[(df_concat.ano_campeonato == 2016) & (df_concat.rodada == 38) & (df_concat.gols_pro != df_concat.gols_pro), 'derrota'] = 1
df_concat.loc[(df_concat.ano_campeonato == 2016) & (df_concat.rodada == 38) & (df_concat.gols_pro != df_concat.gols_pro), ['gols_pro','pontos', 'vitoria','empate']] = 0

display('Após a correção',
        df_concat.loc[(
            (df_concat.rodada == 38) & (df_concat.ano_campeonato == 2016) & ((df_concat.time == 'Chapecoense') | (df_concat.time == 'Atlético-MG')))])

'Antes da correção'

,ano_campeonato,rodada,time,adversário,gols_pro,gols_contra,pontos,vitoria,derrota,empate
4171,2016,38,Chapecoense,Atlético-MG,NaN,NaN,NaN,NaN,NaN,NaN
11771,2016,38,Atlético-MG,Chapecoense,NaN,NaN,NaN,NaN,NaN,NaN


'Após a correção'

,ano_campeonato,rodada,time,adversário,gols_pro,gols_contra,pontos,vitoria,derrota,empate
4171,2016,38,Chapecoense,Atlético-MG,0.0,3.0,0.0,0.0,1.0,0.0
11771,2016,38,Atlético-MG,Chapecoense,0.0,3.0,0.0,0.0,1.0,0.0


In [13]:
df_cumsum = df_concat.sort_values(['ano_campeonato','time','rodada']).copy()
df_cumsum['pontos_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['pontos'].cumsum()
df_cumsum['jogos_acum'] = df_cumsum['vitoria'] + df_cumsum['derrota'] + df_cumsum['empate']
df_cumsum['jogos_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['jogos_acum'].cumsum()
df_cumsum['vitorias_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['vitoria'].cumsum()
df_cumsum['empates_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['empate'].cumsum()
df_cumsum['derrotas_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['derrota'].cumsum()
df_cumsum['gols_pro_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['gols_pro'].cumsum()
df_cumsum['gols_contra_acum'] = df_cumsum.groupby(['time','ano_campeonato'])['gols_contra'].cumsum()
df_cumsum['saldo_gols_acum'] = df_cumsum['gols_pro_acum'] - df_cumsum['gols_contra_acum']

#ajuste para rodadas que ainda não aconteceram
df_cumsum.loc[df_cumsum.gols_pro != df_cumsum.gols_pro, df_cumsum.columns[6:]] = np.nan

df_cumsum

,ano_campeonato,rodada,time,adversário,gols_pro,gols_contra,pontos,vitoria,derrota,empate,pontos_acum,jogos_acum,vitorias_acum,empates_acum,derrotas_acum,gols_pro_acum,gols_contra_acum,saldo_gols_acum
0,2006,1,Atlético-PR,Fluminense,1.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,-1.0
7619,2006,2,Atlético-PR,Santos,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,4.0,-3.0
7620,2006,3,Atlético-PR,Botafogo,4.0,0.0,3.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,2.0,5.0,4.0,1.0
30,2006,4,Atlético-PR,Internacional,1.0,2.0,0.0,0.0,1.0,0.0,3.0,4.0,1.0,0.0,3.0,6.0,6.0,0.0
7647,2006,5,Atlético-PR,Santa Cruz,2.0,1.0,3.0,1.0,0.0,0.0,6.0,5.0,2.0,0.0,3.0,8.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15156,2025,34,Vasco da Gama,Grêmio,0.0,2.0,0.0,0.0,1.0,0.0,42.0,34.0,12.0,6.0,16.0,50.0,51.0,-1.0
15160,2025,35,Vasco da Gama,EC Bahia,0.0,1.0,0.0,0.0,1.0,0.0,42.0,35.0,12.0,6.0,17.0,50.0,52.0,-2.0
7578,2025,36,Vasco da Gama,Internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7589,2025,37,Vasco da Gama,Mirassol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#verificação se todas as rodadas entre 2006 e 2025 apresentam 20 times distintos
df_check = df_cumsum.groupby(['ano_campeonato','rodada']).count().reset_index()[['ano_campeonato','rodada','time']]
df_check.loc[df_check.time != 20]

#verificação ok.

,ano_campeonato,rodada,time


In [15]:
# utilizada a feature "gols_pro" pois mesmo que o jogo esteja cadastrado, se ainda não foi realizado, o gols_pro estará como Nan.
# utilizado o denominador 20 pois apesar de cada rodade ter 10 jogos, cada linha refere-se aos jogos de cada time, logo, mandante e visitante terão linhas exclusivas, por isso 20 linhas por rodada
# a divisao de inteiros por 20 resulta em quantas rodadas completas teremos no df
df_cumsum.dropna(subset='gols_pro').groupby('ano_campeonato')['gols_pro'].count() // 20 

ano_campeonato
2006    38
2007    38
2008    38
2009    38
2010    38
2011    38
2012    38
2013    38
2014    38
2015    38
2016    38
2017    38
2018    38
2019    38
2020    38
2021    38
2022    38
2023    38
2024    38
2025    34
Name: gols_pro, dtype: int64

In [16]:
#identificando os campeões de cada ano
'''
Critérios de desempate do Brasileirão
1º maior número de vitórias.
2º maior saldo de gols.
3º maior número de gols pró
'''

# dados na rodada 19 (1o. turno)
df_rodada19 = df_cumsum.loc[df_cumsum.rodada == 19]
df_rodada19 = df_rodada19.sort_values(['ano_campeonato','pontos_acum','vitorias_acum', 'saldo_gols_acum','gols_pro_acum'])
list_anos_19 = df_rodada19.ano_campeonato.unique().tolist()

#identificando a maior rodada completa em cada ano, já que o ano atual pode estar em andamento e assim a ultima rodada nao será a 38.
# utilizada a feature "gols_pro" pois mesmo que o jogo esteja cadastrado, se ainda não foi realizado, o gols_pro estará como Nan.
# utilizado o denominador 20 pois apesar de cada rodade ter 10 jogos, cada linha refere-se aos jogos de cada time, logo, mandante e visitante terão linhas exclusivas, por isso 20 linhas por rodada
# a divisao de inteiros por 20 resulta em quantas rodadas completas teremos no df
list_anos_38 = df_cumsum.dropna(subset='gols_pro').groupby('ano_campeonato')['rodada'].max().reset_index()
list_anos_38 = list_anos_38.loc[list_anos_38.rodada == 38]
list_anos_38 = list_anos_38.ano_campeonato.unique().tolist()

list_df_rodada38 = []
for ano in list_anos_38:
    list_df_rodada38.append(df_cumsum.loc[(df_cumsum.ano_campeonato == ano) & (df_cumsum.rodada == 38)])

# dados na rodada 38 (final)
df_rodada38 = pd.concat(list_df_rodada38)
df_rodada38 = df_rodada38.sort_values(['ano_campeonato','pontos_acum','vitorias_acum', 'saldo_gols_acum','gols_pro_acum'])

#classificacao por ano
list_df_classificacao_19 = []
list_df_classificacao_38 = []

#rodada 19
for ano in list_anos_19:
    df_19 = df_rodada19.loc[df_rodada19.ano_campeonato == ano][['ano_campeonato','time']]
    df_19['classificacao_1o_turno'] = np.arange(20,0,-1)
    list_df_classificacao_19.append(df_19)
df_classificacao19_ano = pd.concat(list_df_classificacao_19)
# display('19', df_classificacao19_ano)

#rodada 38
for ano in list_anos_38:
    df_38 = df_rodada38.loc[df_rodada38.ano_campeonato == ano][['ano_campeonato','time']]
    df_38['classificacao_final'] = np.arange(20,0,-1)
    list_df_classificacao_38.append(df_38)
df_classificacao38_ano = pd.concat(list_df_classificacao_38)
# display('38', df_classificacao38_ano)

df_completo = df_cumsum.merge(df_classificacao19_ano, on = ['ano_campeonato','time'], how = 'left')
df_completo = df_completo.merge(df_classificacao38_ano, on = ['ano_campeonato','time'], how = 'left')
df_completo

,ano_campeonato,rodada,time,adversário,gols_pro,gols_contra,pontos,vitoria,derrota,empate,pontos_acum,jogos_acum,vitorias_acum,empates_acum,derrotas_acum,gols_pro_acum,gols_contra_acum,saldo_gols_acum,classificacao_1o_turno,classificacao_final
0,2006,1,Atlético-PR,Fluminense,1.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,-1.0,14,13.0
1,2006,2,Atlético-PR,Santos,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,4.0,-3.0,14,13.0
2,2006,3,Atlético-PR,Botafogo,4.0,0.0,3.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,2.0,5.0,4.0,1.0,14,13.0
3,2006,4,Atlético-PR,Internacional,1.0,2.0,0.0,0.0,1.0,0.0,3.0,4.0,1.0,0.0,3.0,6.0,6.0,0.0,14,13.0
4,2006,5,Atlético-PR,Santa Cruz,2.0,1.0,3.0,1.0,0.0,0.0,6.0,5.0,2.0,0.0,3.0,8.0,7.0,1.0,14,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15195,2025,34,Vasco da Gama,Grêmio,0.0,2.0,0.0,0.0,1.0,0.0,42.0,34.0,12.0,6.0,16.0,50.0,51.0,-1.0,16,NaN
15196,2025,35,Vasco da Gama,EC Bahia,0.0,1.0,0.0,0.0,1.0,0.0,42.0,35.0,12.0,6.0,17.0,50.0,52.0,-2.0,16,NaN
15197,2025,36,Vasco da Gama,Internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN
15198,2025,37,Vasco da Gama,Mirassol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN


# Export to parquet

In [17]:
df_completo.to_parquet('../dados/df_completo.parquet')